# 🧪 Laboratório Experimental do SPI (Lex)
### Sistema de Detecção de Máscaras Faciais com MobileNetV2

## 📘 Introdução

Este experimento propõe uma aula prática de Processamento Digital de Imagens (PDI), utilizando um Sistema de Processamento de Imagens (SPI) baseado em inteligência artificial para detectar automaticamente, em tempo real, se uma pessoa está **usando ou não máscara facial**.

O sistema foi desenvolvido com a técnica de *transfer learning* usando a arquitetura **MobileNetV2**, e treinado com imagens de rostos com e sem máscara. A detecção de rosto é feita com a biblioteca **MediaPipe** e a classificação é realizada com um modelo de deep learning.

### 🎯 Objetivo
Demonstrar a aplicação prática do SPI no reconhecimento de máscaras faciais a partir de imagens capturadas pela webcam do usuário, em tempo real, promovendo o entendimento do uso de redes neurais em PDI.


### ✅ Pré-requisitos

- Conta no Google
- Permitir acesso à webcam no navegador
- Conexão à internet
- Dataset pré-carregado no Google Drive


## 🧩 Etapas do Experimento

### 🔹 Parte 2 – Uso Prático pelo Usuário (execução em tempo real)

#### Caso de uso 1 – Captura de imagem com a webcam

**Objetivo:** O usuário tirará uma foto de si mesmo e o sistema indicará se está ou não usando máscara.

**Passos:**

1. Rodar a célula de código "📸 Tirar Foto".
2. Autorizar o uso da webcam.
3. Tirar uma foto.
4. O sistema detectará o(s) rosto(s), classificará cada um e mostrará o resultado com uma borda colorida:
   - Verde: Com máscara
   - Vermelho: Sem máscara


###📦 Célula 1: Instalação de bibliotecas
Essa célula instala as bibliotecas necessárias para que o experimento funcione corretamente:

mediapipe: usada para detecção de rostos com a solução da Google.

tensorflow: utilizado para carregar e rodar o modelo de IA.

opencv-python-headless: versão leve do OpenCV usada para processamento de imagens no Colab.

**Caso ocorra algum erro na execução da célula pressione as teclas Ctrl+M ou clique em Ambiente de execução e reiniciar sessão **

In [1]:
!pip install mediapipe tensorflow opencv-python-headless --quiet

###📚 Célula 2: Importação de bibliotecas
Essa célula importa todas as bibliotecas usadas no sistema:

os: manipulação de arquivos.

cv2: biblioteca do OpenCV para processamento de imagens.

time: usada para pausas no fluxo do experimento.

numpy: manipulação de arrays e matrizes.

matplotlib.pyplot: exibição de gráficos e imagens (pouco usada nesse experimento).

mediapipe: usada para detectar rostos em tempo real.

tensorflow.keras.models.load_model: carrega o modelo treinado.

IPython.display: permite exibir imagens, HTML e JavaScript no notebook.

eval_js e Javascript: usados para capturar imagem da webcam via navegador no Colab.

b64decode: decodifica a imagem capturada da webcam em base64.

In [2]:
import os, cv2, time, numpy as np, matplotlib.pyplot as plt, mediapipe as mp
from tensorflow.keras.models import load_model
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode

###🖼️ Célula 3: Definição do tamanho padrão da imagem
Define a dimensão padrão (100x100 pixels) para as imagens que serão passadas ao modelo. Isso garante que todas as imagens de rostos detectados sejam redimensionadas corretamente para entrada no modelo de IA.

In [3]:
IMG_SIZE = 100

###🤖 Célula 4: Carregamento do modelo treinado
Essa célula carrega o modelo de detecção de máscara facial:

Verifica se o arquivo Modelo.h5 já existe.

Se não existir, faz o download direto do GitHub.

Em seguida, carrega o modelo com load_model para uso posterior na classificação das imagens capturadas.



In [ ]:
modelo_path = "/content/Modelo.h5"
if not os.path.exists(modelo_path):
    print("⬇️ Baixando modelo do GitHub...")
    import urllib.request
    urllib.request.urlretrieve("https://raw.githubusercontent.com/Kinhasne/Face-Mask-Detection/main/Modelo.h5", modelo_path)
modelo = load_model(modelo_path)
print("✅ Modelo carregado com sucesso!")

###📸 Célula 5: Função para capturar foto via webcam
Define a função tirar_foto, responsável por:

Executar um código JavaScript que ativa a câmera do navegador.

Capturar uma imagem e enviá-la de volta ao Python.

Decodificar e salvar essa imagem localmente no Colab.

Exibir a imagem capturada.

Retornar o caminho da imagem para uso posterior.



In [5]:
def tirar_foto(filename='/content/foto.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = '📸 Capturar Foto';
      capture.style.fontSize = '20px';
      capture.style.padding = '10px';
      capture.style.marginTop = '10px';
      div.appendChild(capture);
      const video = document.createElement('video');
      video.style.display = 'block';
      video.style.marginTop = '10px';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      await new Promise((resolve) => capture.onclick = resolve);
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getTracks().forEach(t => t.stop());
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
  ''')
  display(js)
  data = eval_js(f'takePhoto({quality})')
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  print(f"✅ Foto salva: {filename}")
  display(Image(filename))
  return filename

###🧠 Célula 6: Função de detecção de rosto e classificação com o modelo
Essa função realiza as seguintes etapas:

Leitura e conversão da imagem: carrega a imagem capturada e converte de BGR para RGB (formato exigido pelo MediaPipe).

Inicialização do detector MediaPipe: configura o detector de rostos.

Processamento da imagem: detecta rostos na imagem.

Para cada rosto detectado:

Extrai a área do rosto (bounding box).

Redimensiona e normaliza a imagem do rosto.

Passa o rosto pelo modelo para prever se está com ou sem máscara.

Exibe o resultado na imagem com um retângulo colorido e texto explicativo.

Salva e exibe o resultado final.

In [6]:
def detectar_mascara(foto_path):
  imagem = cv2.imread(foto_path)
  imagem_rgb = cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB)
  mp_face = mp.solutions.face_detection
  detector = mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.6)
  resultados = detector.process(imagem_rgb)
  if not resultados.detections:
    print("❌ Nenhum rosto detectado.")
    return
  h, w, _ = imagem.shape
  imagem_resultado = imagem_rgb.copy()
  for i, deteccao in enumerate(resultados.detections):
    box = deteccao.location_data.relative_bounding_box
    x = int(box.xmin * w)
    y = int(box.ymin * h)
    largura = int(box.width * w)
    altura = int(box.height * h)
    pad = 20
    x1, y1 = max(0, x - pad), max(0, y - pad)
    x2, y2 = min(w, x + largura + pad), min(h, y + altura + pad)
    face_crop = imagem_rgb[y1:y2, x1:x2]
    if face_crop.size == 0:
      continue
    face_crop = cv2.resize(face_crop, (IMG_SIZE, IMG_SIZE)) / 255.0
    face_crop = np.expand_dims(face_crop, axis=0)
    pred = modelo.predict(face_crop)[0]
    classe = np.argmax(pred)
    conf = pred[classe] * 100
    status = "Com mascara" if classe == 1 else "Sem mascara"
    cor = (0, 255, 0) if classe == 1 else (255, 0, 0)
    print(f"🔍 Rosto {i+1}: {status} ({conf:.2f}%)")
    cv2.rectangle(imagem_resultado, (x1, y1), (x2, y2), cor, 2)
    cv2.putText(imagem_resultado, f"{status} ({conf:.1f}%)", (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, cor, 2)
  plt.figure(figsize=(10, 8))
  plt.imshow(imagem_resultado)
  plt.axis('off')
  plt.title("Resultado da Detecção")
  plt.show()

### 📸 Célula 7: Captura da Foto
Essas três linhas representam o fluxo completo de capturar uma imagem ao vivo do usuário, esperar que a imagem seja salva corretamente e então classificá-la automaticamente para indicar se há ou não o uso de máscara facial.

**Caso queira testar mais de uma vez, basta executar a célula várias vezes.**

**Tire a foto em um ambiente bem iluminado para melhorar a precisão do resultado.**

In [ ]:
caminho = tirar_foto()
time.sleep(1)
detectar_mascara(caminho)

#📋Face Mask Detection - Questionário
Por favor, execute a célula abaixo e clique no link para acessar nosso formulário. Sua opinião é muito importante para nós!

Agradecemos seu feedback sobre nosso trabalho.


In [ ]:
from IPython.display import display, HTML

link_forms = "https://forms.gle/UCyMqC7hg56JMPJg7"

display(HTML(f'<a href="{link_forms}" target="_blank">Clique aqui para preencher o questionário</a>'))
